In [4]:
import os
import cv2

In [ ]:
input_dir = '../Agriculture-Vision-2021/train/labels'
labels = []
for j in os.listdir(input_dir):
    labels.append(j)
labels

['double_plant',
 'water',
 'storm_damage',
 'planter_skip',
 'weed_cluster',
 'waterway',
 'endrow',
 'drydown',
 'nutrient_deficiency']

In [ ]:
p  = ['../datasets/agriculture_vision/test/images',
'../datasets/agriculture_vision/train/images',
'../datasets/agriculture_vision/train/labels',
'../datasets/agriculture_vision/val/images',
'../datasets/agriculture_vision/val/labels']
for i in p:
    if not os.path.exists(i):
        os.makedirs(i)

## Создание dataset.yaml

In [ ]:
with open('datasets/agriculture_vision/dataset.yaml', 'w') as f:
  f.write('path: agriculture_vision\n')
  f.write('names:\n')
  for j in os.listdir(input_dir):
    f.write(f'- {j}\n')

  f.write('train: train\n')
  f.write('val: val\n')
  f.write('test: test\n')

  

## Перевод масок в контуры для labels

In [6]:
    
def maskToPolygon(input_dir, output_dir):
    for count, j in enumerate(os.listdir(f'{input_dir}/{labels[0]}')):
        filename = j.split('.')[0]

        for i, label in enumerate(labels):
            dir_label = os.path.join(input_dir, label)
            image_path = os.path.join(dir_label, j)

            mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

            H, W = mask.shape
            contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            polygons = []
            for cnt in contours:
                if cv2.contourArea(cnt) > 200:
                    polygon = []
                    for point in cnt:
                        x, y = point[0]
                        polygon.append(x / W)
                        polygon.append(y / H)
                    polygons.append(polygon)


        
            # print the polygons
            with open('{}.txt'.format(os.path.join(output_dir, filename)), 'a') as f:
                for polygon in polygons:
                    for p_, p in enumerate(polygon):
                        if p_ == len(polygon) - 1:
                            f.write('{}\n'.format(p))
                        elif p_ == 0:
                            f.write(str(i) + ' {} '.format(p)) # указание класса 
                        else:
                            f.write('{} '.format(p))

                f.close()

In [ ]:
# labels for train
maskToPolygon(input_dir='../Agriculture-Vision-2021/train/labels', output_dir='../datasets/agriculture_vision/labels/train')

In [ ]:
# labels for validation
maskToPolygon(input_dir='../Agriculture-Vision-2021/val/labels', output_dir='../datasets/agriculture_vision/labels/val')